In [22]:
import os
import shutil
import json
import random
import torch

In [23]:
shutil.rmtree("processed_seqLM_coarse", ignore_errors=True)
os.makedirs("processed_seqLM_coarse", exist_ok=True)
objects_masterlist = open("../../object_list.txt", "r").read().splitlines()
tables = [o for o in objects_masterlist if o.startswith("table")]
supplyshelf = [o for o in objects_masterlist if o.startswith("supply")]
surfaces = tables + supplyshelf
room = [o for o in objects_masterlist if o.startswith("room")]
objects = [o for o in objects_masterlist if o not in tables and o not in supplyshelf and o not in room]

print("Tables:", ', '.join(tables))
print("Supply Shelves:", ', '.join(supplyshelf))
print("Rooms:", ', '.join(room))
print("Objects:", ', '.join(objects))

Tables: tableA, tableB, tableC, tableD, tableE
Supply Shelves: supplyshelf
Rooms: room
Objects: coasters, cup, etchpen, flower, foamball, gemset, glue, googlyeyes, keychain, moss, mossframe, paint, paintbrush, palette, photoframe, plantpot, stones, suncatcher, tissuebox, varnish


In [24]:
static_graph_edges = torch.zeros(len(objects_masterlist), len(objects_masterlist), dtype=torch.float32)
active_edges_mask = torch.zeros_like(static_graph_edges, dtype=torch.float32)
for surface in tables + supplyshelf:
    static_graph_edges[objects_masterlist.index(surface), objects_masterlist.index("room")] = 1.0
    for object in objects:
        active_edges_mask[objects_masterlist.index(object), objects_masterlist.index(surface)] = 1.0


In [ ]:
common_data = {
    'dt': 5.0,
    'start_time': 0.0,
    'end_time': 60*3,
    'node_classes': objects_masterlist,
    'node_categories': ['Rooms' if obj in room else 'Furniture' if obj in surfaces else 'placable_objects' for obj in objects_masterlist],
    'edge_keys': ["ON"],
    'static_nodes': [obj for obj in objects_masterlist if obj not in objects],
    'static_node_categories': ["Rooms", "Furniture"],
    'dataset_type': "RoboCraft",
    'n_stations': 5,
}
json.dump(common_data, open("processed_seqLM_coarse/common_data.json", "w"))

common_edge_data = {}
common_edge_data['home_graph'] = static_graph_edges.clone()
for obj in objects:
    common_edge_data['home_graph'][objects_masterlist.index(obj), objects_masterlist.index('room')] = 1.0
common_edge_data['nonstatic_edges'] = active_edges_mask.clone()
common_edge_data['seen_edges'] = torch.zeros_like(active_edges_mask, dtype=torch.float32)
torch.save(common_edge_data, "processed_seqLM_coarse/common_edge_data.pt")

common_embedding_map = {}


In [26]:
nodes = torch.tensor([i for i in range(len(objects_masterlist))], dtype=torch.float32)
times = torch.arange(common_data['start_time'], common_data['end_time'], common_data['dt'], dtype=torch.float32)
activity = torch.zeros((len(times), 5), dtype=torch.float32)

In [ ]:
def initialize_scene_graph_edges():
    scene_graph = static_graph_edges.clone()
    for object in objects:
        number_of_instances = random.randint(1, 5)
        for _ in range(number_of_instances):
            scene_graph[objects_masterlist.index(object), objects_masterlist.index(random.choice(surfaces))] = 1.0
    return scene_graph

def initialize_activity_labels():
    num_people = torch.zeros((5), dtype=torch.float32)
    for i in range(5):
        num_people[i] = random.randint(0, 10)  # Random number of people for each activity
    return num_people

def sample_next_scene_graph_edges(scene_graph):
    next_scene_graph = scene_graph.clone()
    for object in objects:
        if random.random() < 0.1:  # 50% chance to add a new edge
            next_scene_graph[objects_masterlist.index(object), objects_masterlist.index(random.choice(surfaces))] = 1.0
        current_edges = next_scene_graph[objects_masterlist.index(object), :].nonzero(as_tuple=False).squeeze(dim=-1)
        if len(current_edges) > 0 and random.random() < 0.1:  # 10% chance to remove an edge
            next_scene_graph[objects_masterlist.index(object), current_edges[random.randint(0, len(current_edges) - 1)]] = 0.0
    return next_scene_graph

def sample_next_activity_labels(activity):
    next_activity = activity.clone()
    for i in range(5):
        if random.random() < 0.1:  # 10% chance to change the number of people
            next_activity[i] = random.randint(0, 10)
    return next_activity


In [28]:
for dirname, num_samples in [["test", 10], ["train", 50]]:
    os.makedirs(f"processed_seqLM_coarse/{dirname}", exist_ok=True)
    for sample_num in range(num_samples):
        scene_graph_list = [initialize_scene_graph_edges()]
        activity_list = [initialize_activity_labels()]
        for time in times[1:]:
            next_scene_graph = sample_next_scene_graph_edges(scene_graph_list[-1])
            scene_graph_list.append(next_scene_graph)
            next_activity = sample_next_activity_labels(activity_list[-1])
            activity_list.append(next_activity)
        
        # Save the scene graph and activity data
        torch.save({
            'nodes': nodes,
            'edges': torch.stack(scene_graph_list, dim=0),
            'times': times,
            'active_edges': active_edges_mask,
            'activity': torch.stack(activity_list, dim=0),
        }, f"processed_seqLM_coarse/{dirname}/{sample_num:03d}.pt")